In [206]:
import requests
import pandas as pd

#uncomment this line on error 
!pip install bs4
from bs4 import BeautifulSoup
#uncomment this line on error 
!pip install wikipedia
import wikipedia as wp
print('Libraries imported.')

Libraries imported.


In [207]:
website_url = requests.get('https://es.wikipedia.org/wiki/Madrid#Distritos').text
#website_url

Reading the web page.

In [208]:
#convert the request
soup = BeautifulSoup(website_url, 'html.parser')
#print(soup.prettify())               #Uncomment if you want to see the data

The page has a section with an ordered list of districts. Their neighborhoods are listed next to each district.

In [209]:
listas = soup.find_all('ol')

The list of Districts corresponds to the first list that appears on the page.

In [210]:
listas[0] 

<ol><li><b><a href="/wiki/Centro_(Madrid)" title="Centro (Madrid)">Centro</a>.</b> <a href="/wiki/Palacio_(Madrid)" title="Palacio (Madrid)">Palacio</a>, <a href="/wiki/Embajadores_(Madrid)" title="Embajadores (Madrid)">Embajadores</a>, <a href="/wiki/Cortes_(Madrid)" title="Cortes (Madrid)">Cortes</a>, <a href="/wiki/Justicia_(Madrid)" title="Justicia (Madrid)">Justicia</a>, <a class="mw-redirect" href="/wiki/Malasa%C3%B1a" title="Malasaña">Universidad</a> y <a href="/wiki/Sol_(Madrid)" title="Sol (Madrid)">Sol</a>.</li>
<li><b><a href="/wiki/Arganzuela" title="Arganzuela">Arganzuela</a>.</b> <a class="mw-redirect" href="/wiki/Imperial_(Arganzuela)" title="Imperial (Arganzuela)">Imperial</a>, <a class="mw-redirect" href="/wiki/Las_Acacias" title="Las Acacias">Acacias</a>, <a class="mw-redirect" href="/wiki/La_Chopera" title="La Chopera">La Chopera</a>, <a href="/wiki/Legazpi_(Madrid)" title="Legazpi (Madrid)">Legazpi</a>, <a class="mw-redirect" href="/wiki/Las_Delicias_(Madrid)" title

We access one of the linked pages and retrieve its content to read the coordinates.

In [211]:
urlwikipage ='https://es.wikipedia.org' + '/wiki/Arapiles_(Madrid)'
wikipage_url = requests.get(urlwikipage).text
wikipage_url
#convert the request
soupwiki = BeautifulSoup(wikipage_url, 'html.parser')

The location, latitude and longitude, is in a link to another page. We locate the links that have the same class.

<span class="latitude">40.470196, </span><span class="longitude">-3.58489</span>

In [212]:
celdaCoords=soupwiki.find_all('a',{"class":"external text"})


In [213]:
celdaCoords[1]

<a class="external text" href="http://tools.wmflabs.org/geohack/geohack.php?language=es&amp;pagename=Arapiles_(Madrid)&amp;params=40.43416667_N_-3.70777778_E_type:city"><span class="geo-default"><span class="geo-dms" title="Mapas, fotos y otros datos de 40°26′03″N 3°42′28″O"><span class="latitude">40°26′03″N </span><span class="longitude">3°42′28″O</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-nondefault"><span class="geo-dec" title="Mapas, fotos y otros datos de 40.43416667 -3.70777778"><span class="geo"><span class="latitude">40.43416667, </span><span class="longitude">-3.70777778</span></span></span></span></a>

In [214]:
latitud=float(celdaCoords[1].find_all('span',{"class":"latitude"})[1].text.split(',')[0])
print(latitud)

40.43416667


In [215]:
longitud=float(celdaCoords[1].find_all('span',{"class":"longitude"})[1].text.split(',')[0])
print(longitud)

-3.70777778


Once we have identified the processes of the data source pages, we will recover the data of each District and its neighborhoods iterating from the initial page to create a dataframe with the coordinates.

In [216]:
rows=[]
for distritos in listas[0].find_all('li'):
    distri=0
    for distrito in distritos.find_all('a'):
        print(distrito.get('title'))
        urlwikipage ='https://es.wikipedia.org' + distrito.get('href')
        wikipage_url = requests.get(urlwikipage).text
        soupwiki = BeautifulSoup(wikipage_url, 'html.parser')
        celdaCoords=soupwiki.find_all('a',{"class":"external text"})
        try:
            latitud=float(celdaCoords[0].find_all('span',{"class":"latitude"})[1].text.split(',')[0])
            longitud=float(celdaCoords[0].find_all('span',{"class":"longitude"})[1].text.split(',')[0])
        except:
            try:
                latitud=float(celdaCoords[1].find_all('span',{"class":"latitude"})[1].text.split(',')[0])
                longitud=float(celdaCoords[1].find_all('span',{"class":"longitude"})[1].text.split(',')[0])
            except:
                latitud=0
                longitud=0
        if distri==0:
            #print('Distrito:',distrito.get('href'),distrito.get('title'))
            rows.append([distrito.get('title'),distrito.get('href'),'Distrito', latitud, longitud])
        else:
            #print('Barrio:',distrito.get('href'),distrito.get('title'))
            rows.append([distrito.get('title'),distrito.get('href'),'Barrio', latitud, longitud])
        distri +=1

Centro (Madrid)
Palacio (Madrid)
Embajadores (Madrid)
Cortes (Madrid)
Justicia (Madrid)
Malasaña
Sol (Madrid)
Arganzuela
Imperial (Arganzuela)
Las Acacias
La Chopera
Legazpi (Madrid)
Las Delicias (Madrid)
Palos de Moguer (Madrid)
Atocha (Madrid)
Retiro (Madrid)
Pacífico (Madrid)
Adelfas
Estrella (Madrid)
Ibiza (Madrid)
Jerónimos (Madrid)
Niño Jesús (Madrid)
Salamanca (Madrid)
Recoletos (Madrid)
Goya (Madrid)
Fuente del Berro (Madrid)
Guindalera
Lista (Madrid)
Castellana (Madrid)
Chamartín
Colonia de El Viso
Prosperidad (Madrid)
Ciudad Jardín (Madrid)
Hispanoamérica (Madrid)
Nueva España (Madrid)
Castilla (Madrid)
Tetuán (Madrid)
Bellas Vistas
Cuatro Caminos (Madrid)
Castillejos (Madrid)
Almenara (Madrid)
Valdeacederas
Berruguete (Madrid)
Chamberí
Gaztambide (Madrid)
Arapiles (Madrid)
Trafalgar (Madrid)
Almagro (Madrid)
Ríos Rosas (Madrid)
Vallehermoso (Madrid)
Fuencarral-El Pardo
El Pardo (Madrid)
Fuentelarreina
Peñagrande
Barrio del Pilar (Madrid)
La Paz (Madrid)
Valverde (Madrid)
Mir

In [217]:
listado=pd.DataFrame(rows) 
listado.columns=['Name','link','Type','latitude','longitude']
print(listado.shape)
listado=listado[listado['latitude']!=0]
print(listado.shape)

(152, 5)
(151, 5)


We save the dataframe in a csv file for later use.

In [218]:
listado.to_csv('barrios.csv',index=False)

Once downloaded to the csv, the data can be recovered:

In [219]:
listado = pd.read_csv('barrios.csv')
listado.head()

,Name,link,Type,latitude,longitude
0,Centro (Madrid),/wiki/Centro_(Madrid),Distrito,40.415347,-3.707371
1,Palacio (Madrid),/wiki/Palacio_(Madrid),Barrio,40.415000,-3.713333
2,Embajadores (Madrid),/wiki/Embajadores_(Madrid),Barrio,40.408889,-3.699722
3,Cortes (Madrid),/wiki/Cortes_(Madrid),Barrio,40.414167,-3.698056
4,Justicia (Madrid),/wiki/Justicia_(Madrid),Barrio,40.423889,-3.696389


Madrid location:

In [220]:
latitudMadrid=40.418889
longitudMadrid=-3.691944

In [221]:
listadobarrios=listado[listado['Type']=='Barrio'].reset_index()
listadoDistritos=listado[listado['Type']=='Distrito'].reset_index()

We prepare to visualize the location of each District, Neighborhood and charging station in Madrid

In [222]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_madrid = folium.Map(location=[latitudMadrid, longitudMadrid], zoom_start=11)

We load the placeholders of the Districts ...

In [223]:
# add Distrito markers to map
for lat, lng, label in zip(listadoDistritos['latitude'], listadoDistritos['longitude'], listadoDistritos['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF5733',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  

We load the placeholders of each neighborhood ...

In [224]:
# add markers to map
for lat, lng, label in zip(listadobarrios['latitude'], listadobarrios['longitude'], listadobarrios['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    

In [225]:
map_madrid

## We retrieve the location data of the charging stations.

We downloaded the KML file from Google Earth with the export of the search "estación carga vehículo eléctrico madrid".

Leemos el archivo desde la ruta local donde lo hemos guardado:

In [226]:
import xml.etree.ElementTree as ET

pathtokml='estacion de carga de vehiculos electrico madrid.kml'
tree=ET.parse(pathtokml)

Para extraer los datos de ubicación de cada estación de carga localizamos el elemento {http://www.opengis.net/kml/2.2}Placemark que contiene los datos de latitud y longitud.

In [227]:
places=tree.findall('.//{http://www.opengis.net/kml/2.2}Placemark')

Creamos un diccionario con los elementos Placemark y sus propiedades.

In [228]:
places[0].findall('.//{http://www.opengis.net/kml/2.2}coordinates')[0].text.split(',')[0]
places[0].findall('.//{http://www.opengis.net/kml/2.2}name')[0].text.split(',')[0]


'Punto de recarga de Vehículos Eléctricos'

In [229]:
elementos=[]
for place in places:
    latitud=float(place.findall('.//{http://www.opengis.net/kml/2.2}coordinates')[0].text.split(',')[1])
    longitud=float(place.findall('.//{http://www.opengis.net/kml/2.2}coordinates')[0].text.split(',')[0])
    nombre=place.findall('.//{http://www.opengis.net/kml/2.2}name')[0].text.split(',')[0]
    direccion=place.findall('.//{http://www.opengis.net/kml/2.2}address')[0].text.split(',')[0]
    elementos.append([nombre,direccion,latitud,longitud])

stations=pd.DataFrame(elementos) 
stations.columns=['Name','Address','latitude','longitude']
stations

,Name,Address,latitude,longitude
0,Punto de recarga de Vehículos Eléctricos,28046 Madrid,40.431880,-3.689155
1,Enchufauto,Calle de Diego de León,40.434853,-3.678520
2,Recarga eléctrica de coches y motos,Calle de Alfonso XII,40.419270,-3.688927
3,Recarga eléctrica de coches,Paseo de la Castellana,40.458343,-3.689345
4,Recarga eléctrica de coches,Calle de los Chulapos,40.407397,-3.720168
5,Punto de Carga para Vehiculo Eléctrico Sta. En...,Calle de Sta Engracia,40.438250,-3.700276
6,Recarga eléctrica de coches y motos,Calle de Goya,40.424696,-3.671250
7,Punto de Carga Vehículos Eléctricos,Parking ifema,40.464751,-3.618176
8,Punto de recarga Vehículo Eléctrico,Calle de San Bernardo,40.427178,-3.706448


Cargamos los marcadores de posición de cada estación de carga...

In [230]:
# add markers to map
for lat, lng, label in zip(stations['latitude'], stations['longitude'], stations['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#4FFF33',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    

In [231]:
map_madrid

# Explore and cluster the neighborhoods in Madrid

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [232]:
VERSION = '20180605' # Foursquare API version

CLIENT_ID = 'myclientid'
CLIENT_SECRET = 'myClientSecret'
CLIENT_ID = 'DPQJAJTCGDD4023TRPCVLR5FLNABK1LEUIJCDVA4AQTQEYQ3'
CLIENT_SECRET = 'TFW0DW2XXGO1DRA2JD3OUNQV1PONCDMMXAN1KSKF3R3GAEWI'


#### Let's explore the first neighborhood in our dataframe.

In [233]:
listadobarrios.loc[0,'Name']

'Palacio (Madrid)'

Get the neighborhood's latitude and longitude values.

In [234]:
neighborhood_latitude = listadobarrios.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = listadobarrios.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = listadobarrios.loc[0, 'Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Palacio (Madrid) are 40.415, -3.71333333.


In [235]:
#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [236]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    500, 
    100)


url

'https://api.foursquare.com/v2/venues/explore?&client_id=DPQJAJTCGDD4023TRPCVLR5FLNABK1LEUIJCDVA4AQTQEYQ3&client_secret=TFW0DW2XXGO1DRA2JD3OUNQV1PONCDMMXAN1KSKF3R3GAEWI&v=20180605&ll=40.415,-3.71333333&radius=500&limit=100'

In [237]:
results = requests.get(url).json()

We know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [238]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [239]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Santa Iglesia Catedral de Santa María la Real ...,Church,40.415767,-3.714516
1,Cervecería La Mayor,Beer Bar,40.415218,-3.712194
2,Plaza de La Almudena,Plaza,40.416320,-3.713777
3,Mercado Jamón Iberico,Market,40.415309,-3.711633
4,Taberna Rayuela,Tapas Restaurant,40.413179,-3.713496


In [240]:
nearby_venues

,name,categories,lat,lng
0,Santa Iglesia Catedral de Santa María la Real ...,Church,40.415767,-3.714516
1,Cervecería La Mayor,Beer Bar,40.415218,-3.712194
2,Plaza de La Almudena,Plaza,40.416320,-3.713777
3,Mercado Jamón Iberico,Market,40.415309,-3.711633
4,Taberna Rayuela,Tapas Restaurant,40.413179,-3.713496
...,...,...,...,...
95,Casa Revuelta,Spanish Restaurant,40.413832,-3.708065
96,Petit Palace Mayor Plaza,Hotel,40.415892,-3.709029
97,Restaurante Posada del Dragón,Restaurant,40.413080,-3.708521
98,La Antoñita,Spanish Restaurant,40.413085,-3.708551


And how many venues were returned by Foursquare?

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

## Explore Neighborhoods in Madrid

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [241]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    LIMIT=limit
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *madrid_venues*.

In [242]:
madrid_venues = getNearbyVenues(names=listadobarrios['Name'],
                                   latitudes=listadobarrios['latitude'],
                                   longitudes=listadobarrios['longitude']
                                  )

Palacio (Madrid)
Embajadores (Madrid)
Cortes (Madrid)
Justicia (Madrid)
Malasaña
Sol (Madrid)
Imperial (Arganzuela)
Las Acacias
La Chopera
Legazpi (Madrid)
Las Delicias (Madrid)
Palos de Moguer (Madrid)
Atocha (Madrid)
Pacífico (Madrid)
Adelfas
Estrella (Madrid)
Ibiza (Madrid)
Jerónimos (Madrid)
Niño Jesús (Madrid)
Recoletos (Madrid)
Goya (Madrid)
Fuente del Berro (Madrid)
Guindalera
Lista (Madrid)
Castellana (Madrid)
Colonia de El Viso
Prosperidad (Madrid)
Ciudad Jardín (Madrid)
Hispanoamérica (Madrid)
Nueva España (Madrid)
Castilla (Madrid)
Bellas Vistas
Cuatro Caminos (Madrid)
Castillejos (Madrid)
Almenara (Madrid)
Valdeacederas
Berruguete (Madrid)
Gaztambide (Madrid)
Arapiles (Madrid)
Trafalgar (Madrid)
Almagro (Madrid)
Ríos Rosas (Madrid)
Vallehermoso (Madrid)
El Pardo (Madrid)
Fuentelarreina
Peñagrande
Barrio del Pilar (Madrid)
La Paz (Madrid)
Valverde (Madrid)
Mirasierra
Casa de Campo
Argüelles (Madrid)
Ciudad Universitaria de Madrid
Valdezarza
Valdemarín
El Plantío
Aravaca
Los 

In [243]:
madrid_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Palacio (Madrid),40.415000,-3.713333,Santa Iglesia Catedral de Santa María la Real ...,40.415767,-3.714516,Church
1,Palacio (Madrid),40.415000,-3.713333,Cervecería La Mayor,40.415218,-3.712194,Beer Bar
2,Palacio (Madrid),40.415000,-3.713333,Plaza de La Almudena,40.416320,-3.713777,Plaza
3,Palacio (Madrid),40.415000,-3.713333,Mercado Jamón Iberico,40.415309,-3.711633,Market
4,Palacio (Madrid),40.415000,-3.713333,Taberna Rayuela,40.413179,-3.713496,Tapas Restaurant
...,...,...,...,...,...,...,...
3727,Corralejos,40.464444,-3.590000,Parque Bahía de Almería,40.463568,-3.591574,Park
3728,Corralejos,40.464444,-3.590000,L'Andecha Cafetería Pastelería,40.462978,-3.591601,Café
3729,Corralejos,40.464444,-3.590000,Restaurante Asiático Hong Yun,40.462819,-3.591709,Asian Restaurant
3730,Corralejos,40.464444,-3.590000,"Alimentación, pan, bebida y frutos secos",40.462925,-3.592227,Food & Drink Shop


In [244]:
madrid_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelfas,34,34,34,34,34,34
Aeropuerto (Madrid),16,16,16,16,16,16
Alameda de Osuna,25,25,25,25,25,25
Almagro (Madrid),73,73,73,73,73,73
Almenara (Madrid),4,4,4,4,4,4
...,...,...,...,...,...,...
Valverde (Madrid),1,1,1,1,1,1
Ventas (Madrid),23,23,23,23,23,23
Villaverde Alto (Madrid),5,5,5,5,5,5


Guardamos el dataframe en un archivo para uso posterior.

In [245]:
madrid_venues.to_csv('madrid_venues.csv',index=False)

#### Now write the code to run the above function on each station and create a new dataframe called *madrid_station_venues*.

In [246]:
madrid_station_venues = getNearbyVenues(names=stations['Name'],
                                   latitudes=stations['latitude'],
                                   longitudes=stations['longitude']
                                  )

Punto de recarga de Vehículos Eléctricos
Enchufauto
Recarga eléctrica de coches y motos
Recarga eléctrica de coches
Recarga eléctrica de coches
Punto de Carga para Vehiculo Eléctrico Sta. Engracia- José Abascal
Recarga eléctrica de coches y motos
Punto de Carga Vehículos Eléctricos
Punto de recarga Vehículo Eléctrico


In [247]:
madrid_station_venues.to_csv('madrid_stations_venues.csv',index=False)

In [248]:
madrid_station_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Enchufauto,82,82,82,82,82,82
Punto de Carga Vehículos Eléctricos,13,13,13,13,13,13
Punto de Carga para Vehiculo Eléctrico Sta. Engracia- José Abascal,64,64,64,64,64,64
Punto de recarga Vehículo Eléctrico,100,100,100,100,100,100
Punto de recarga de Vehículos Eléctricos,100,100,100,100,100,100
Recarga eléctrica de coches,91,91,91,91,91,91
Recarga eléctrica de coches y motos,162,162,162,162,162,162
